In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.features import CustomIcon

In [69]:
df = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스 5조/2.데이터/서울시 행정동 중심좌표.CSV',encoding='cp949')

df.rename(columns = {'Longitude\t' : 'Longitude'}, inplace = True)
df.rename(columns = {'\tLatitude' : 'Latitude'}, inplace = True)


df.head()

,시군구,읍면동,Latitude,Longitude
0,종로구,청운효자동,37.584009,126.970626
1,종로구,사직동,37.575408,126.965944
2,종로구,삼청동,37.590758,126.980996
3,종로구,부암동,37.594768,126.965574
4,종로구,평창동,37.613029,126.974485


- 서울시 지도 json 받아오기

In [4]:
geo_str = json.load(open("/content/drive/MyDrive/멀티캠퍼스 5조/2.데이터/json/seoul_municipalities_geo_simple.json", encoding = 'utf-8'))

In [18]:
map = folium.Map(location = [37.5502, 126.982], zoom_start=11)
marker_cluster = MarkerCluster().add_to(map) 

for i in range(len(df)):
    icon1 = CustomIcon(
    '/content/drive/MyDrive/멀티캠퍼스 5조/기타 자료(이미지 파일 등)/red marker.png',
    icon_size=(10,10)
    )

    folium.Marker(
        location = [df.loc[i,'Latitude'], df.loc[i,'Longitude']],
        icon = icon1,
        tooltip = df.loc[i, '읍면동']
        ).add_to(map)


def style_function(feature):
    default_style = {
        'opacity': 1,
        'fillColor': 'white',
        'color': 'darkred',
        'weight': 2.5
        }
    return default_style

folium.GeoJson(geo_str,style_function = style_function).add_to(map)

map

http://tmapapi.sktelecom.com/main.html#web/usecase/UseCaseOptimization

https://developers.google.com/maps/documentation/directions/overview?hl=ko

In [21]:
cdf = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스 5조/2.데이터/콜택시 차고지 csv파일.CSV',encoding='cp949')
cdf.head()

,차고지명,소재지,Latitude,Longitude,주차대수,유무료
0,개화산,강서구 방화동 845,37.572192,126.805159,20,무료
1,고척,구로구 고척로45길 39,37.505901,126.853182,10,유료
2,구파발,은평구 진관동 66-30,37.637967,126.919285,10,무료
3,남부여성(민),금천구 시흥4동 산139-2,37.463078,126.905934,12,유료
4,대림운동장,영등포구 대림3동 780,37.499611,126.894810,10,유료


In [28]:
df.head()

,시군구,읍면동,Latitude,Longitude
0,종로구,청운효자동,37.584009,126.970626
1,종로구,사직동,37.575408,126.965944
2,종로구,삼청동,37.590758,126.980996
3,종로구,부암동,37.594768,126.965574
4,종로구,평창동,37.613029,126.974485


In [25]:
!pip install haversine


In [66]:
df.head()

,시군구,읍면동,Latitude,Longitude,min_dis
0,종로구,청운효자동,37.584009,126.970626,
1,종로구,사직동,37.575408,126.965944,
2,종로구,삼청동,37.590758,126.980996,
3,종로구,부암동,37.594768,126.965574,
4,종로구,평창동,37.613029,126.974485,


In [74]:
from tqdm import tqdm

from haversine import haversine
alldis=[]
for i in tqdm(range(len(df))):
    dis=[]
    dong= (df.loc[i][2], df.loc[i][3])
    for e in range(len(cdf)):
        car = (cdf.loc[e][2], cdf.loc[e][3])
        dis.append(haversine(dong, car, unit = 'km'))
    alldis.append(min(dis))


100%|██████████| 891/891 [00:09<00:00, 95.04it/s]


In [75]:
len(alldis)

891

In [77]:
df.insert(4,"차고지와의 최단거리",alldis,True)

In [78]:
df.head()

,시군구,읍면동,Latitude,Longitude,차고지와의 최단거리
0,종로구,청운효자동,37.584009,126.970626,1.207005
1,종로구,사직동,37.575408,126.965944,0.890775
2,종로구,삼청동,37.590758,126.980996,1.918177
3,종로구,부암동,37.594768,126.965574,2.483024
4,종로구,평창동,37.613029,126.974485,3.624372


In [79]:
df.to_csv("행정동-차고지와의 최단거리 추가.csv", mode='w')
